In [ ]:
import cv2
import pytesseract
import numpy as np
import dlib
from imutils import face_utils
from playsound import playsound

In [ ]:


# # Initialize the camera
# cap = cv2.VideoCapture(0)

# # Initialize the face detector and landmark predictor
# detector = dlib.get_frontal_face_detector()
# predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

# # Initialize state variables
# sleep = 0
# drowsy = 0
# active = 0
# status = ""
# color = (0, 0, 0)

# # Path to the alarm sound file
# alarm_sound_path = "alarm_sound.mp3"

# # Define compute and blinked functions
# def compute(ptA, ptB):
#     dist = np.linalg.norm(ptA - ptB)
#     return dist

# def blinked(a, b, c, d, e, f):
#     up = compute(b, d) + compute(c, e)
#     down = compute(a, f)
#     ratio = up / (2.0 * down)
    
#     if ratio > 0.25:
#         return 2
#     elif 0.21 <= ratio <= 0.25:
#         return 1
#     else:
#         return 0

# while True:
#     # Read a frame from the camera
#     ret, frame = cap.read()
#     gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

#     # Detect faces in the frame
#     faces = detector(gray)

#     # Process each detected face
#     for face in faces:
#         x1 = face.left()
#         y1 = face.top()
#         x2 = face.right()
#         y2 = face.bottom()

#         # Copy the frame and draw a rectangle around the face
#         face_frame = frame.copy()
#         cv2.rectangle(face_frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

#         # Get landmarks and convert them to numpy array
#         landmarks = predictor(gray, face)
#         landmarks = face_utils.shape_to_np(landmarks)

#         # Check for blinks
#         left_blink = blinked(landmarks[36], landmarks[37], landmarks[38], landmarks[41], landmarks[40], landmarks[39])
#         right_blink = blinked(landmarks[42], landmarks[43], landmarks[44], landmarks[47], landmarks[46], landmarks[45])
        
#         # Update state based on blink detection
#         if left_blink == 0 or right_blink == 0:
#             sleep += 1
#             drowsy = 0
#             active = 0
#             if sleep > 6:
#                 status = "SLEEPING !!!"
#                 color = (255, 0, 0)
                
#                 # Play the alarm sound when the status is "SLEEPING !!!"
#                 playsound(alarm_sound_path)
#         elif left_blink == 1 or right_blink == 1:
#             sleep = 0
#             active = 0
#             drowsy += 1
#             if drowsy > 6:
#                 status = "Drowsy !"
#                 color = (0, 0, 255)
#         else:
#             drowsy = 0
#             sleep = 0
#             active += 1
#             if active > 6:
#                 status = "Active :)"
#                 color = (0, 255, 0)
                
#         # Display the status text
#         cv2.putText(frame, status, (100, 100), cv2.FONT_HERSHEY_SIMPLEX, 1.2, color, 3)
        
#         # Draw landmarks on the face frame
#         for n in range(0, 68):
#             (x, y) = landmarks[n]
#             cv2.circle(face_frame, (x, y), 1, (255, 255, 255), -1)

#     # Display the frames
#     cv2.imshow("Frame", frame)
#     cv2.imshow("Result of detector", face_frame)

#     # Break the loop if 'q' key is pressed
#     if cv2.waitKey(1) & 0xFF == ord('q'):
#         break

# # Release the camera and destroy all windows
# cap.release()
# cv2.destroyAllWindows()


In [ ]:
# Initialize the video capture object (0 represents the default webcam)
cap = cv2.VideoCapture(0)

# Initialize the face detector and facial landmarks predictor from dlib
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

# Define status variables for the blink detection
sleep = 0
drowsy = 0
active = 0
status = ""
color = (0, 0, 0)

# Define functions for computing distances and checking blinks
def compute(ptA, ptB):
    dist = np.linalg.norm(ptA - ptB)
    return dist

def blinked(a, b, c, d, e, f):
    up = compute(b, d) + compute(c, e)
    down = compute(a, f)
    ratio = up / (2.0 * down)
    
    # Check for blinks based on ratio
    if ratio > 0.25:
        return 2
    elif 0.21 <= ratio <= 0.25:
        return 1
    else:
        return 0

# Define a function to perform OCR on a frame and print the extracted text
def perform_ocr(frame):
    # Convert the frame to grayscale for better OCR accuracy
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Apply thresholding (binarization)
    _, binary_frame = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY)

    # Use Tesseract to extract text from the preprocessed frame
    extracted_text = pytesseract.image_to_string(binary_frame, lang='eng')

    # Perform OCR using pytesseract on the grayscale frame
    # text = pytesseract.image_to_string(gray)

    # Print the extracted text in the console
    print("Extracted Text:", extracted_text)

# Continuously read frames from the video capture object
while True:
    ret, frame = cap.read()
    
    # Check if the frame is read successfully
    if not ret:
        print("Error: Failed to read frame.")
        break

    # Convert the frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces in the grayscale frame
    faces = detector(gray)
    face_frame = frame.copy()
    
    # Process each detected face
    for face in faces:
        x1 = face.left()
        y1 = face.top()
        x2 = face.right()
        y2 = face.bottom()

        # Draw rectangles around detected faces
        cv2.rectangle(face_frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

        # Get facial landmarks
        landmarks = predictor(gray, face)
        landmarks = face_utils.shape_to_np(landmarks)

        # Process eye blinks
        left_blink = blinked(landmarks[36], landmarks[37], landmarks[38], landmarks[41], landmarks[40], landmarks[39])
        right_blink = blinked(landmarks[42], landmarks[43], landmarks[44], landmarks[47], landmarks[46], landmarks[45])

        # Judge the eye blinks
        if left_blink == 0 or right_blink == 0:
            sleep += 1
            drowsy = 0
            active = 0
            if sleep > 6:
                status = "SLEEPING"
                color = (255, 0, 0)
                
                # Play alarm sound when the status is "SLEEPING"
                playsound("alarm.wav")
                
        elif left_blink == 1 or right_blink == 1:
            sleep = 0
            active = 0
            drowsy += 1
            if drowsy > 6:
                status = "Drowsy"
                color = (0, 0, 255)
        else:
            drowsy = 0
            sleep = 0
            active += 1
            if active > 6:
                status = "Active"
                color = (0, 255, 0)

        # Display status on the frame
        cv2.putText(frame, status, (100, 100), cv2.FONT_HERSHEY_SIMPLEX, 1.2, color, 3)

        # Draw facial landmarks on the face frame
        for n in range(0, 68):
            (x, y) = landmarks[n]
            cv2.circle(face_frame, (x, y), 1, (255, 255, 255), -1)
    
    # Perform OCR and print the extracted text
    perform_ocr(frame)

    # Show the original frame
    cv2.imshow("Frame", frame)

    # Show the frame with detected faces and landmarks
    if face_frame is not None:
        cv2.imshow("Face Frame", face_frame)

    # Check for 'q' key to exit the loop
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture object and close OpenCV windows
cap.release()
cv2.destroyAllWindows()
